以下代码部分参考：

https://www.cjavapy.com/article/746/

https://blog.csdn.net/qq_29678299/article/details/103509618

词频分析

In [1]:
# if you haven't download the "jieba" package, plz run this line
#pip install jieba

In [56]:
import jieba.analyse as analyse
import jieba
import re
import pandas as pd
import numpy as np
from string import digits

In [22]:
# index_col=0: turn to the first column to index
cn_data_comp = pd.read_csv("../data/China Data Content Competition.csv", encoding="utf-8",index_col=0)
cn_data_comp.head()

,Competition,Award Category,Year,Item,Original Name,Name,Data Processing,Brief Comments,Data Source Label Location,Original Content,Translated Text,Manual,Machine,Rank,Link
0,China Data Content Competition,Best data content,2021,Gold Award,解构藏文,Deconstruct Tibetan,Some signs,Blur processing,Text,本作品将从数据的视角去解读藏文的文化价值。,This work will interpret the cultural value of...,None,Some signs,3.0,https://v.qq.com/x/page/t3263rg503z.html
1,China Data Content Competition,Best data content,2021,Gold Award,解构藏文,Deconstruct Tibetan,Clearly marked,Clearly stated,Picture annotation,我们这么做是为了通过数据可视化，克服文化障碍井增强藏汉文化关联。,We do this to overcome cultural barriers and e...,None,Clear,4.0,NaN
2,China Data Content Competition,Best data content,2021,Gold Award,解构藏文,Deconstruct Tibetan,Some signs,Blur processing,Text,这是个探索藏族语言文字的数字工具，就可以匹配到相应的藏文招牌走进藏地生活，你只需要画两笔去了...,This is a digital tool for exploring the Tibet...,Some signs,Some signs,3.5,NaN
3,China Data Content Competition,Best data content,2021,Gold Award,解构藏文,Deconstruct Tibetan,Some signs,Blur processing,Text,我们通过时间轴还原了老师在传习所里一天的生活，他每天大都分时间都花费在藏文书法及相关知识的学习上。,We restored the teacher's day-to-day life in t...,Some signs,Some signs,3.5,NaN
4,China Data Content Competition,Best data content,2021,Gold Award,解构藏文,Deconstruct Tibetan,Clearly marked,Clearly stated,Text,我们将诗歌中的词语进行拆分，并对书写时采集到的脑电波数据与肌电数据进行图形可视化呈现。,We split the words in the poems and visualized...,None,Clear,4.0,NaN


In [23]:
cn_data_comp = cn_data_comp.rename(columns=lambda x:x.replace(" ",""))
cn_data_comp.head()

,Competition,AwardCategory,Year,Item,OriginalName,Name,DataProcessing,BriefComments,DataSourceLabelLocation,OriginalContent,TranslatedText,Manual,Machine,Rank,Link
0,China Data Content Competition,Best data content,2021,Gold Award,解构藏文,Deconstruct Tibetan,Some signs,Blur processing,Text,本作品将从数据的视角去解读藏文的文化价值。,This work will interpret the cultural value of...,None,Some signs,3.0,https://v.qq.com/x/page/t3263rg503z.html
1,China Data Content Competition,Best data content,2021,Gold Award,解构藏文,Deconstruct Tibetan,Clearly marked,Clearly stated,Picture annotation,我们这么做是为了通过数据可视化，克服文化障碍井增强藏汉文化关联。,We do this to overcome cultural barriers and e...,None,Clear,4.0,NaN
2,China Data Content Competition,Best data content,2021,Gold Award,解构藏文,Deconstruct Tibetan,Some signs,Blur processing,Text,这是个探索藏族语言文字的数字工具，就可以匹配到相应的藏文招牌走进藏地生活，你只需要画两笔去了...,This is a digital tool for exploring the Tibet...,Some signs,Some signs,3.5,NaN
3,China Data Content Competition,Best data content,2021,Gold Award,解构藏文,Deconstruct Tibetan,Some signs,Blur processing,Text,我们通过时间轴还原了老师在传习所里一天的生活，他每天大都分时间都花费在藏文书法及相关知识的学习上。,We restored the teacher's day-to-day life in t...,Some signs,Some signs,3.5,NaN
4,China Data Content Competition,Best data content,2021,Gold Award,解构藏文,Deconstruct Tibetan,Clearly marked,Clearly stated,Text,我们将诗歌中的词语进行拆分，并对书写时采集到的脑电波数据与肌电数据进行图形可视化呈现。,We split the words in the poems and visualized...,None,Clear,4.0,NaN


In [24]:
# missing value processing
cn_data_comp = cn_data_comp.dropna()

In [26]:
cn_data_comp_content = cn_data_comp.loc[:,["OriginalContent"]]
cn_data_comp_content

,OriginalContent
0,本作品将从数据的视角去解读藏文的文化价值。
13,在烂尾楼投诉留言中，能识别出建楼、购房、停工或承诺交房年份的共有2829条。如果用这个时间点...
22,以 “拐卖妇女”作为检索案由，共得到涉及拐卖妇女的有效裁判文书616份。
25,杀子案件的女性相对年轻，集中于26-29岁，杀夫案件的女性多为中年女性，集中于42-45岁。
28,福利姬的推特账户粉丝数量整理分析得出
35,全国肿瘤登记中心的统计数据显示，2000 年，我国 20 - 39 岁年龄组的肿瘤发病率为 ...
41,"作品通过可视化叙事方法,以动态信息设计为主，"
42,《“建党百年”信息可视化设计》作品运用信息设计、平面设计、动态影像等多元化的设计手法。
43,辽代车马仪仗制度信息图设计主要视觉形象为信息插图。
44,信息图表根据表述女性对象的年龄阶段、社会阶级、社会背景、社会地位以及所创作诗歌的诗人背景、词...


In [27]:
lines = cn_data_comp.OriginalContent.values.tolist()
content = "".join(lines)
content

"本作品将从数据的视角去解读藏文的文化价值。在烂尾楼投诉留言中，能识别出建楼、购房、停工或承诺交房年份的共有2829条。如果用这个时间点与投诉者留言的年份相减，我们可以大致估计出某处楼房的烂尾时长…以 “拐卖妇女”作为检索案由，共得到涉及拐卖妇女的有效裁判文书616份。杀子案件的女性相对年轻，集中于26-29岁，杀夫案件的女性多为中年女性，集中于42-45岁。福利姬的推特账户粉丝数量整理分析得出全国肿瘤登记中心的统计数据显示，2000 年，我国 20 - 39 岁年龄组的肿瘤发病率为 39.22/10 万。2013 年，这一数字则增长到了 70.01 / 10 万，增长了近 80%。如果以 2013 年的人口统计数据为基数进行计算，我国每年都会有 30 万名年轻人罹患癌症。作品通过可视化叙事方法,以动态信息设计为主，《“建党百年”信息可视化设计》作品运用信息设计、平面设计、动态影像等多元化的设计手法。辽代车马仪仗制度信息图设计主要视觉形象为信息插图。信息图表根据表述女性对象的年龄阶段、社会阶级、社会背景、社会地位以及所创作诗歌的诗人背景、词派等角度说明女性地位在中国传统社会的变化与社会对女性形象的影响。借助信息可视化设计的方式使其以现代化的规觉表现方式呈现于大众视好之中。滑动鼠标滚轮获取全部图表，点击可获得更佳的交互体验。Data Speaks is a six-episode series that analyzes China's economic growth from different perspectives. The sixth one looks at Wuhan's economic recovery through satellite maps. \xa0我们把这些坐标放在了一张地图上，地图上的每个亮点都代表着数据库记录的一次恐怖袭击。我们以每个项目首次进入奥运会的成绩为标准值，将历届奥运会冠军成绩相连，绘制出一根根成绩线。纵坐标表示选手在其比赛项目上的长度、高度或速度成绩，位置越高则成绩提升越快。恰逢春运期间，人们流动频繁，疫情随着“人传人”就这么扩散开来。这里的每棵树代表一簇有传染关系的患者们。树干的两端连着两名患者，上面移动的光点是传染方向，没有光点则说明这两名患者只是接触、但不一定谁传给谁。一共有1191条树干带着光点，635条树干没有光

In [28]:
lines = cn_data_comp.OriginalContent.values.tolist()
content = "".join(lines)
content_nonum = content.translate(digits)
print(content_nonum)

本作品将从数据的视角去解读藏文的文化价值。在烂尾楼投诉留言中，能识别出建楼、购房、停工或承诺交房年份的共有2829条。如果用这个时间点与投诉者留言的年份相减，我们可以大致估计出某处楼房的烂尾时长…以 “拐卖妇女”作为检索案由，共得到涉及拐卖妇女的有效裁判文书616份。杀子案件的女性相对年轻，集中于26-29岁，杀夫案件的女性多为中年女性，集中于42-45岁。福利姬的推特账户粉丝数量整理分析得出全国肿瘤登记中心的统计数据显示，2000 年，我国 20 - 39 岁年龄组的肿瘤发病率为 39.22/10 万。2013 年，这一数字则增长到了 70.01 / 10 万，增长了近 80%。如果以 2013 年的人口统计数据为基数进行计算，我国每年都会有 30 万名年轻人罹患癌症。作品通过可视化叙事方法,以动态信息设计为主，《“建党百年”信息可视化设计》作品运用信息设计、平面设计、动态影像等多元化的设计手法。辽代车马仪仗制度信息图设计主要视觉形象为信息插图。信息图表根据表述女性对象的年龄阶段、社会阶级、社会背景、社会地位以及所创作诗歌的诗人背景、词派等角度说明女性地位在中国传统社会的变化与社会对女性形象的影响。借助信息可视化设计的方式使其以现代化的规觉表现方式呈现于大众视好之中。滑动鼠标滚轮获取全部图表，点击可获得更佳的交互体验。Data Speaks is a six-episode series that analyzes China's economic growth from different perspectives. The sixth one looks at Wuhan's economic recovery through satellite maps.  我们把这些坐标放在了一张地图上，地图上的每个亮点都代表着数据库记录的一次恐怖袭击。我们以每个项目首次进入奥运会的成绩为标准值，将历届奥运会冠军成绩相连，绘制出一根根成绩线。纵坐标表示选手在其比赛项目上的长度、高度或速度成绩，位置越高则成绩提升越快。恰逢春运期间，人们流动频繁，疫情随着“人传人”就这么扩散开来。这里的每棵树代表一簇有传染关系的患者们。树干的两端连着两名患者，上面移动的光点是传染方向，没有光点则说明这两名患者只是接触、但不一定谁传给谁。一共有1191条树干带着光点，635条树干没有光点。20

中文词频分析中常用的通用词表有：
+ 中文停用词表
+ 哈工大停用词表
+ 百度停用词表
+ 四川大学机器智能实验室停用词库
现使用将以上停用词表进行合并的停用词表（共计2311个），来自@elephantnose(https://gitcode.net/mirrors/elephantnose/characters?utm_source=csdn_github_accelerator。)



In [29]:
# 指定停用词
jieba.analyse.set_stop_words('../data/stop_words.txt')

In [38]:
#1､基于 TF-IDF 算法的关键词抽取的方法
words_TF = analyse.extract_tags(content, topK=200, allowPOS=('ns', 'n', 'vn', 'v'))
print(" ".join(words_TF))

信息 设计 官网 可视化 光点 女性 成绩 数据 拐卖妇女 树干 省份 流调 育婴室 作品 社会 发电 建党 动态 患者 留言 图表 传染 影像 商场 确诊 年份 多元化 肿瘤 病例 公布 项目 收集 统计数据 手法 投诉者 食源性 地图 案件 网站 记录 形象 补充 出建 案由 杀子 杀夫 了近 词派 规觉 非水 相减 资料 背景 生物质 年龄组 统计 纵坐标 年龄阶段 标准值 小时数 处方药 烂尾 鼠标 烂尾楼 地位 滚轮 恐怖袭击 罹患 更佳 预计 代表 京东 交房 生命周期 方式 传人 检索 粉丝 药店 比赛项目 滑动 暴发 政府 视角 车马 交互 增长势头 停工 数据库 裁判 发病率 招股书 点击 仪仗 插图 叙事 选手 癌症 传给 增长 楼房 坐标 增设 绘制 福利 疫情 投诉 视觉 情况 细分 简化 表述 下发 基数 购房 亮点 识别 冠军 解读 扩散 市场份额 年报 开来 体验 诗歌 大众 文书 长度 监测 借助 登记 年轻人 官方 共计 份额 零售 获取 诗人 阶级 修改 现代化 期限 流动 事项 市场 账户 数字 承诺 享受 接触 文字 整理 创作 角度 扩张 呈现 疾病 医药 共有 新增 总体 通知 高度 估计 新闻 对象 放在 涉及 小时 事件 位置 突破 计算 提升 速度 传统 方向 趋势 人口 数量 报告 制度 导致 发布 文化 利用 方法 风险 价值 分析 中心 全国 变化 关系 超过 显示 表现 发现 发生 时间


In [39]:
#2､基于 TextRank 算法的关键词抽取
words_textrank = analyse.textrank(content, topK=200, allowPOS=('ns', 'n', 'vn', 'v'))
print(" ".join(words_textrank))

信息 设计 成绩 社会 数据 女性 发电 作品 可视化 市场 留言 肿瘤 官网 统计数据 项目 年份 光点 图表 地图 趋势 楼房 增长 拐卖妇女 传染 方式 背景 分析 地位 生命周期 统计 代表 登记 病例 省份 疫情 扩散 速度 生物质 承诺 零售 整理 传人 全国 动态 医药 数据库 患者 网站 扩张 收集 手法 政府 处方药 购房 食源性 确诊 疾病 中心 暴发 发生 获取 利用 补充 形象 更佳 长度 投诉者 烂尾楼 数量 突破 市场份额 价值 风险 停工 情况 树干 多元化 交房 高度 绘制 招股书 表现 案由 涉及 位置 解读 裁判 记录 公布 新增 商场 共计 事件 亮点 基数 识别 药店 增设 增长势头 鼠标 文书 滚轮 账户 粉丝 小时数 育婴室 资料 关系 选手 官方 呈现 投诉 比赛项目 规觉 相减 小时 创作 烂尾 数字 文化 案件 总体 新闻 对象 监测 表述 年龄阶段 体验 估计 了近 年报 交互 共有 点击 细分 预计 纵坐标 现代化 方向 影像 传统 中国 变化 放在 显示 车马 插图 坐标 文字 仪仗 流调 诗人 影响 报告 滑动 词派 冠军 杀子 流动 开来 诗歌 方法 视觉 检索 癌症 借助 通知 事项 角度 恐怖袭击 导致 享受 出建 阶级 大众 修改 计算 简化 人口 制度 年龄组 发病率 时间 叙事 发布 罹患 年轻人 建党 期限 发现 视角 杀夫 标准值 提升 非水 发展 接触 京东


主题模型

In [19]:
#pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [7]:
import gensim
from gensim import corpora, models, similarities

In [48]:
with open("stop_words.txt", "r") as f:
    data = f.readlines()
    stopword = [i.rstrip().split('\n') for i in data]
    print(stopword)

[['"'], ['#'], ['$'], ['&'], ["'"], ['('], [')'], ['*'], ['+'], [','], ['-'], ['.'], ['/'], ['0'], ['1'], ['2'], ['3'], ['4'], ['5'], ['6'], ['7'], ['8'], ['9'], [':'], [';'], ['<'], ['='], ['>'], ['?'], ['@'], ['['], [']'], ['_'], ['}'], ['·'], ['×'], ['Δ'], ['Ψ'], ['γ'], ['μ'], ['φ'], ['В'], ['—'], ['‘'], ['’'], ['“'], ['”'], ['℃'], ['Ⅲ'], ['↑'], ['→'], ['≈'], ['①'], ['②'], ['③'], ['④'], ['⑤'], ['⑥'], ['⑦'], ['⑧'], ['⑨'], ['⑩'], ['■'], ['▲'], ['、'], ['。'], ['〉'], ['《'], ['》'], ['」'], ['『'], ['』'], ['【'], ['】'], ['〔'], ['〕'], ['㈧'], ['一'], ['上'], ['下'], ['不'], ['与'], ['且'], ['个'], ['临'], ['为'], ['乃'], ['么'], ['之'], ['乎'], ['乘'], ['也'], ['了'], ['于'], ['些'], ['亦'], ['人'], ['今'], ['仍'], ['从'], ['他'], ['以'], ['们'], ['任'], ['会'], ['但'], ['何'], ['你'], ['使'], ['依'], ['俺'], ['倘'], ['借'], ['像'], ['儿'], ['兮'], ['其'], ['内'], ['再'], ['冒'], ['冲'], ['几'], ['凡'], ['凭'], ['则'], ['别'], ['到'], ['即'], ['却'], ['去'], ['又'], ['及'], ['另'], ['只'], ['叫'], ['可'], ['各'], ['同'], ['后'], ['向'], ['吓'], ['吗'], ['吧']

In [65]:
sentences = []
for line in lines:
    line = ''.join([i for i in line if not i.isdigit()])# remove the digital number in the content
    try:
        segs = jieba.lcut(line)
        segs = filter(lambda x:len(x)>1, segs) # select the word whose lenth is larger than 1
        segs = filter(lambda x:x not in stopword, segs) # remove the stop words
        #segs = filter(lambda x:isinstance(x,int) or , segs)
        sentences.append(list(segs))
    except Exception:
        print(line)
        continue


dic = corpora.Dictionary(sentences)
corpus = [dic.doc2bow(sentence) for sentence in sentences]

# LDA modeling
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dic, num_topics=20)

#print all the topics
for topic in lda.print_topics(num_topics=20, num_words=8):
    print("Topic",topic[0],":")
    print(topic[1])
    print("------------------------------------------------------")
    
#lda.print_topics(num_topics=20, num_words=8)

Topic 0 :
0.037*"图表" + 0.037*"体验" + 0.037*"点击" + 0.037*"更佳" + 0.037*"鼠标" + 0.037*"滑动" + 0.037*"全部" + 0.037*"获取"
------------------------------------------------------
Topic 1 :
0.041*"成绩" + 0.034*"信息" + 0.020*"奥运会" + 0.018*"食源性" + 0.018*"风险" + 0.018*"监测" + 0.018*"车马" + 0.018*"事件"
------------------------------------------------------
Topic 2 :
0.058*"光点" + 0.046*"树干" + 0.043*"患者" + 0.031*"传染" + 0.031*"没有" + 0.030*"两名" + 0.017*"说明" + 0.017*"上面"
------------------------------------------------------
Topic 3 :
0.003*"健康" + 0.003*"关于" + 0.003*"享受" + 0.003*"促进" + 0.003*"下发" + 0.003*"再生能源" + 0.003*"随着" + 0.003*"利用"
------------------------------------------------------
Topic 4 :
0.003*"健康" + 0.003*"关于" + 0.003*"享受" + 0.003*"促进" + 0.003*"下发" + 0.003*"再生能源" + 0.003*"随着" + 0.003*"利用"
------------------------------------------------------
Topic 5 :
0.027*"预计" + 0.025*"市场" + 0.024*"线上" + 0.017*"健康" + 0.017*"最大" + 0.016*"药店" + 0.016*"港交所" + 0.016*"京东"
----------------------------------------------

以上代码部分参考：

https://www.cjavapy.com/article/746/

https://blog.csdn.net/qq_29678299/article/details/103509618